read_sample lee una muestra a la vez

Espera hasta timeout para devolver un error si no logra leer la muestra con el mensaje:
SampleAcquisitionTimeout: **Time out occurred while waiting for a start byte**

Da la impresión de que no hay buffer, si leo el puerto serie a tiempos superiores al intervalo entre muestras se pierden muestras. OpenBCI lo dice a través del byte de parada que no será 0x0C y tenemos acceso porque en la estructura de datos que envía por cada muestra el campo Valid = False

**IMPORTANTE** El ID de muestra es de 8 bits, se reinicia a 0 cada 256 muestras
** Secuencia**
cyton.read_sample -> cyton.read_packet -> cyton.wait_start_byte -> pyserial.Serial.read(1)

pyserial.Serial.read(size) se duerme hasta que entra valor por puerto serie o ocurre timeout y devuelve la cantidad de bytes que haya logrado leer

dentro de cyton.wait_start_byte se lanza exception SampleAcquisitionTimeout cuando el pyserial.Serial.read() ha llegado al timeout y por lo tanto no devuelve ningún valor

Estrategias:
1. Probar con solamente una muestra
2. 




In [10]:
import time
import pandas as pd
from openbci_interface import Cyton
from openbci_interface.util import list_devices
from serial import Serial
import pickle

# from saccrec.consts import DEBUG

"""
def list_ports():
    if DEBUG:
        return [
            '/dev/ttyUSB0',
            '/dev/ttyUSB1',
        ]
    return [port for port in list_devices()]
"""
def init_board():
    sample_rate = 250
    puerto = '/dev/ttyUSB1'
    port = Serial(
        port=puerto,
        baudrate=115200,
        timeout=2
    )

    print("configurando placa...\n")
    board = Cyton(port)
    board.set_board_mode('default')
    board.set_sample_rate(sample_rate)
    board.configure_channel(1, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(2, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(3, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(4, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(5, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(6, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(7, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(8, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(9, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(10, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(11, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(12, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(13, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(14, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(15, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.configure_channel(16, power_down='ON', gain=24, input_type='NORMAL', bias=0, srb2=0, srb1=1)
    board.disable_channel(5)
    board.disable_channel(6)
    board.disable_channel(7)
    board.disable_channel(8)
    board.disable_channel(9)
    board.disable_channel(10)
    board.disable_channel(11)
    board.disable_channel(12)    
    board.disable_channel(13)
    board.disable_channel(14)
    board.disable_channel(15)
    board.disable_channel(16)
    
    print(board.get_config())  
    return board
def read_board(board):
    lista = []
    cnt = 0
    board.start_streaming()
    for x in range(500):
        sample = board.read_sample()
        cnt += 1
        lista.append(sample)
        # time.sleep(0.85 / sample_rate)
    return lista, cnt

In [14]:
board = init_board()

configurando placa...

{'board_mode': 'default', 'sample_rate': 250, 'channels': [{'enabled': None, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': None, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': None, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': None, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': False, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': False, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': False, 'parameters': {'power_down': 'ON', 'gain': 24, 'input_type': 'NORMAL', 'bias': 0, 'srb2': 0, 'srb1': 1}}, {'enabled': False, 'parameters': {'power_down':

In [12]:
data,cnt = read_board(board)
board.stop_streaming()
board.terminate()
"""
fichero = open('datos.pickle', 'wb') 
pickle.dump(lista,fichero)
fichero.close()
"""

Skipped 30 bytes at start.
/home/r/.local/lib/python3.7/site-packages/openbci_interface/cyton.py:55: UserWarning: Stop Byte is 0. Formats other than 0xC0 (Standard with accel) is not implemented.
  '(Standard with accel) is not implemented.' % stop_byte)
Skipped 1 bytes at start.
/home/r/.local/lib/python3.7/site-packages/openbci_interface/cyton.py:55: UserWarning: Stop Byte is 102. Formats other than 0xC0 (Standard with accel) is not implemented.
  '(Standard with accel) is not implemented.' % stop_byte)
Skipped 28 bytes at start.


"\nfichero = open('datos.pickle', 'wb') \npickle.dump(lista,fichero)\nfichero.close()\n"

In [13]:
board.channel_configs

[Channel: 0 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 1 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 2 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 3 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 4 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 5 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 6 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 7 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 8 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0, SRB1: 1,
 Channel: 9 (Disabled), POWER_DOWN: ON, GAIN: 24, INPUT_TYPE: NORMAL, BIAS_SET: 0, SRB2: 0,

In [24]:
data

[{'packet_id': 0,
  'eeg': [187500.0, 187500.0, 187500.0, 187500.0, 0.0, 0.0, 0.0, 0.0],
  'aux': [0.03, -0.008, 0.494],
  'valid': True,
  'timestamp': 1565709333.9951181},
 {'packet_id': 1,
  'eeg': [187500.0, 187500.0, 187500.0, 187500.0, 0.0, 0.0, 0.0, 0.0],
  'aux': [0.0, 0.0, 0.0],
  'valid': True,
  'timestamp': 1565709333.995576},
 {'packet_id': 2,
  'eeg': [187500.0, 187500.0, 187500.0, 187500.0, 0.0, 0.0, 0.0, 0.0],
  'aux': [0.0, 0.0, 0.0],
  'valid': True,
  'timestamp': 1565709333.9959974},
 {'packet_id': 3,
  'eeg': [187500.0, 187500.0, 187500.0, 187500.0, 0.0, 0.0, 0.0, 0.0],
  'aux': [0.0, 0.0, 0.0],
  'valid': True,
  'timestamp': 1565709333.9964132},
 {'packet_id': 4,
  'eeg': [187500.0, 187500.0, 187500.0, 187500.0, 0.0, 0.0, 0.0, 0.0],
  'aux': [0.0, 0.0, 0.0],
  'valid': True,
  'timestamp': 1565709333.9968286},
 {'packet_id': 5,
  'eeg': [187500.0, 187500.0, 187500.0, 187500.0, 0.0, 0.0, 0.0, 0.0],
  'aux': [0.0, 0.0, 0.0],
  'valid': True,
  'timestamp': 15657093